<a href="https://colab.research.google.com/github/sangramkesharidash/Google-Colab-files/blob/main/EQUITYDATA_Screener_EDA_Using_PySpark_SQL_30_APR_2025_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#SPARK SQL

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()
print('Spark Version: {}'.format(spark.version))

Spark Version: 3.5.5


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import os
from google.colab import drive

# Check if Google Drive is mounted; mount if not
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted.")

# Define file path and output folder
file_path = "/content/drive/My Drive/equitydata-screener-all-stocks/equitydata-screener-all-stocks-20250430.csv"
print(file_path)
!ls -ltr $file_path

Mounted at /content/drive
/content/drive/My Drive/equitydata-screener-all-stocks/equitydata-screener-all-stocks-20250430.csv
ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/equitydata-screener-all-stocks/equitydata-screener-all-stocks-20250430.csv': No such file or directory


In [4]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

# Loading the data
dataset = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the data in the above file using the below command
#dataset.show(5)

# It's good practice to stop the SparkSession when you're done
# spark.stop()

In [5]:
from pyspark.sql import SparkSession

# Define file path (replace with your actual file path if needed)
#file_path = '/content/drive/My Drive/equitydata-screener-all-stocks/equitydata-screener-all-stocks-20250428.csv'

# Create a SparkSession
spark = SparkSession.builder.appName("IndustryPEAnalysis").getOrCreate()

# Loading the data
dataset = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first 5 rows of the data
print("First 5 rows of the dataset:")
#dataset.show(5)

# Print the schema of the DataFrame
print("Schema of the dataset:")
#dataset.printSchema()

# Register the DataFrame as a temporary SQL view
dataset.createOrReplaceTempView("equity_data")

# Execute SQL query to calculate the average Industry PE for each industry
result_df = spark.sql("""
SELECT
        Industry,
        CAST(SUM(`Market Capitalization`) AS BIGINT) as Total_Market_Capitalization_CR,
        COUNT(Name) as Total_Stocks,
        ROUND(AVG(`Industry PE`), 0) as Average_Industry_PE,
        CAST(ROUND(SUM(CASE WHEN `Price to Earning` < `Industry PE` THEN 1 ELSE 0 END) * 100 / COUNT(Name), 0) AS INT) as Percentage_Below_Industry_PE
    FROM
        equity_data
    GROUP BY
        Industry
    ORDER BY
        Total_Market_Capitalization_CR DESC
""")

# Show the result
print("\n Industry Analysis :")
result_df.show(truncate=False)

# It's good practice to stop the SparkSession when you're done
spark.stop()

First 5 rows of the dataset:
Schema of the dataset:

 Industry Analysis :
+----------------------------------------------+------------------------------+------------+-------------------+----------------------------+
|Industry                                      |Total_Market_Capitalization_CR|Total_Stocks|Average_Industry_PE|Percentage_Below_Industry_PE|
+----------------------------------------------+------------------------------+------------+-------------------+----------------------------+
|Finance & Investments                         |3810188                       |556         |20.0               |35                          |
|Banks - Private Sector                        |3792444                       |29          |12.0               |52                          |
|Computers - Software - Large                  |3051179                       |10          |30.0               |50                          |
|Refineries                                    |2344604                   

In [6]:
from pyspark.sql import SparkSession

# Define file path (replace with your actual file path if needed)
#file_path = '/content/drive/My Drive/equitydata-screener-all-stocks/equitydata-screener-all-stocks-20250430.csv'

# Create a SparkSession
spark = SparkSession.builder.appName("IndustryPEAnalysis").getOrCreate()

# Loading the data
dataset = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first 5 rows of the data
print("First 5 rows of the dataset:")
#dataset.show(5)

# Print the schema of the DataFrame
print("Schema of the dataset:")
#dataset.printSchema()

# Register the DataFrame as a temporary SQL view
dataset.createOrReplaceTempView("equity_data")

# Execute SQL query to calculate the average Industry PE for each industry
result_df = spark.sql("""
SELECT
        Industry,
        CAST(SUM(`Market Capitalization`) AS BIGINT) as Total_Market_Capitalization_CR,
        COUNT(Name) as Total_Stocks,
        ROUND(AVG(`Industry PE`), 0) as Average_Industry_PE,
        CAST(ROUND(SUM(CASE WHEN `Price to Earning` < `Industry PE` THEN 1 ELSE 0 END) * 100 / COUNT(Name), 0) AS INT) as Percentage_Below_Industry_PE
    FROM
        equity_data
    GROUP BY
        Industry
    ORDER BY
        Total_Market_Capitalization_CR DESC
""")

# Show the result
print("\n Industry Analysis :")
result_df.show(truncate=False)

# It's good practice to stop the SparkSession when you're done
spark.stop()

First 5 rows of the dataset:
Schema of the dataset:

 Industry Analysis :
+----------------------------------------------+------------------------------+------------+-------------------+----------------------------+
|Industry                                      |Total_Market_Capitalization_CR|Total_Stocks|Average_Industry_PE|Percentage_Below_Industry_PE|
+----------------------------------------------+------------------------------+------------+-------------------+----------------------------+
|Finance & Investments                         |3810188                       |556         |20.0               |35                          |
|Banks - Private Sector                        |3792444                       |29          |12.0               |52                          |
|Computers - Software - Large                  |3051179                       |10          |30.0               |50                          |
|Refineries                                    |2344604                   

In [7]:
from pyspark.sql import SparkSession

# Define file path (replace with your actual file path if needed)
file_path = '/content/drive/My Drive/equitydata-screener-all-stocks/equitydata-screener-all-stocks-20250430.csv'

# Create a SparkSession
spark = SparkSession.builder.appName("IndustryAnalysis").getOrCreate()

# Loading the data
dataset = spark.read.csv(file_path, header=True, inferSchema=True)

# Register the DataFrame as a temporary SQL view
dataset.createOrReplaceTempView("equity_data")

# Get all unique industries
industries_df = spark.sql("SELECT DISTINCT Industry FROM equity_data")
industries = [row.Industry for row in industries_df.collect()]

# Iterate over each industry and print the analysis
for industry in industries:
    query = f"""
        SELECT
            Name,
            `High price` as `52w High Rs.`,
            `Low price` as `52w Low`,
            round((`High price` - `Current Price`) / `High price` * 100, 1) as `Down %`,
            `Current Price`,
            `DMA 200`,
            `DMA 50`,
            Industry,
            `RSI`,
            `MACD`,
            `Price to Earning` as PE_Ratio,
            `Industry PE`,
            round(`Market Capitalization` / Sales, 1) as PS_Ratio,
            round(`Market Capitalization` / (`Sales latest quarter` * 4), 1) as `PS_LastQtr`,
            round(`Sales latest quarter` / (`Profit after tax latest quarter` * 4), 1) as `LastQtr_PAT_Pct`
        FROM
            equity_data
        WHERE
            Industry = '{industry}' AND `Current Price` < `DMA 200`
    """
    result_df = spark.sql(query)
    print(f"\n Industry: {industry}")
    result_df.show(truncate=False)

# It's good practice to stop the SparkSession when you're done
spark.stop()


 Industry: Personal Care - Multinational
+----------------+------------+-------+------+-------------+--------+--------+-----------------------------+-----+------+--------+-----------+--------+----------+---------------+
|Name            |52w High Rs.|52w Low|Down %|Current Price|DMA 200 |DMA 50  |Industry                     |RSI  |MACD  |PE_Ratio|Industry PE|PS_Ratio|PS_LastQtr|LastQtr_PAT_Pct|
+----------------+------------+-------+------+-------------+--------+--------+-----------------------------+-----+------+--------+-----------+--------+----------+---------------+
|Colgate-Palmoliv|3893.0      |2311.5 |33.5  |2587.4       |2741.69 |2547.95 |Personal Care - Multinational|53.94|55.82 |48.13   |54.01      |11.6    |12.0      |1.1            |
|Gillette India  |10699.0     |6548.1 |24.9  |8036.5       |8353.59 |8186.6  |Personal Care - Multinational|48.99|11.7  |55.24   |54.01      |9.4     |9.5       |1.4            |
|Hind. Unilever  |3035.0      |2136.0 |22.8  |2342.1       |241